# Syncing data to JanusGraph benchmarks no debug+schema+indexes

## Used configuration:

* without debug mode
* with schema
* with indexes

In [1]:
from thoth.storages.graph import GraphDatabase
from thoth.storages import SolverResultsStore
from thoth.storages import AnalysisResultsStore

from thoth.lab import obtain_location
from thoth.lab import GraphQueryResult as gqr

#graph_db = GraphDatabase.create(obtain_location('thoth-sbu-janusgraph-test1', verify=False, only_netloc=True), port=80)
graph_db = GraphDatabase.create('localhost', port=8182)
solver_results = SolverResultsStore('thoth-sbu', host=obtain_location('thoth-upshift-ceph', verify=False))
analysis_results = AnalysisResultsStore('thoth-sbu', host=obtain_location('thoth-upshift-ceph', verify=False))

graph_db.connect()
solver_results.connect()
analysis_results.connect()

## Place for experiments:

These are benchmarks when running JanusGraph on a local machine. **Note** the debug mode of JanusGraph was turned **off**.

In [2]:
from thoth.common import init_logging

# Let's keep info about enabling/disabling caches.
init_logging({'thoth.storages.graph.utils': 'DEBUG'})

## Syncing image analysis results - without adapter cache

In [3]:
from thoth.lab import GraphQueryResult as gqr

# Erase content of the graph database:
gqr(graph_db.g.V().drop().next()).result
assert gqr(graph_db.g.V().count().next()).result == 0

In [4]:
#list(analysis_results.get_document_listing())
analysis_document = analysis_results.retrieve_document('fridex-thoth-package-extract-dash-4f47l')

In [5]:
%%time
%env THOTH_STORAGES_DISABLE_CACHE=1

graph_db.sync_analysis_result(analysis_document)

DEBUG:thoth.storages.graph.utils:Disabling edge graph cache
DEBUG:thoth.storages.graph.utils:Disabling vertex graph cache


env: THOTH_STORAGES_DISABLE_CACHE=1
CPU times: user 20.1 s, sys: 920 ms, total: 21.1 s
Wall time: 1min 15s


In [6]:
print(f"Number of vertexes present in graph: {gqr(graph_db.g.V().count().next()).result:d}")

Number of vertexes present in graph: 2827


In [7]:
print(f"Number of edges present in graph: {gqr(graph_db.g.E().count().next()).result:d}")

Number of edges present in graph: 9515


## Syncing image analysis results - with adapter cache

In [8]:
gqr(graph_db.g.V().drop().next())
assert gqr(graph_db.g.V().count().next()).result == 0

In [9]:
%%time
%env THOTH_STORAGES_DISABLE_CACHE=0

graph_db.sync_analysis_result(analysis_document)

DEBUG:thoth.storages.graph.utils:Enabling edge graph cache
DEBUG:thoth.storages.graph.utils:Enabling vertex graph cache


env: THOTH_STORAGES_DISABLE_CACHE=0
CPU times: user 28.4 s, sys: 727 ms, total: 29.1 s
Wall time: 1min 6s


In [10]:
print(f"Number of vertexes present in graph: {gqr(graph_db.g.V().count().next()).result:d}")

Number of vertexes present in graph: 2827


In [11]:
print(f"Number of edges present in graph: {gqr(graph_db.g.E().count().next()).result:d}")

Number of edges present in graph: 9515


## Syncing solver results - without adapter cache

In [12]:
gqr(graph_db.g.V().drop().next())
assert gqr(graph_db.g.V().count().next()).result == 0

In [13]:
solver_document = solver_results.retrieve_document('fridex-thoth-solver-fc27-thoth-solver-fc27-92qtv')

In [14]:
%%time
%env THOTH_STORAGES_DISABLE_CACHE=1

graph_db.sync_solver_result(solver_document)

DEBUG:thoth.storages.graph.utils:Disabling edge graph cache
DEBUG:thoth.storages.graph.utils:Disabling vertex graph cache


env: THOTH_STORAGES_DISABLE_CACHE=1
CPU times: user 30.4 s, sys: 1.52 s, total: 31.9 s
Wall time: 1min 35s


In [15]:
print(f"Number of vertexes present in graph: {gqr(graph_db.g.V().count().next()).result:d}")

Number of vertexes present in graph: 653


In [16]:
print(f"Number of edges present in graph: {gqr(graph_db.g.E().count().next()).result:d}")

Number of edges present in graph: 9075


## Syncing solver results - with adapter cache

In [17]:
gqr(graph_db.g.V().drop().next())
assert gqr(graph_db.g.V().count().next()).result == 0

In [18]:
%%time
%env THOTH_STORAGES_DISABLE_CACHE=0

graph_db.sync_solver_result(solver_document)

DEBUG:thoth.storages.graph.utils:Enabling edge graph cache
DEBUG:thoth.storages.graph.utils:Enabling vertex graph cache


env: THOTH_STORAGES_DISABLE_CACHE=0
CPU times: user 25.8 s, sys: 728 ms, total: 26.5 s
Wall time: 1min 5s


In [19]:
print(f"Number of vertexes present in graph: {gqr(graph_db.g.V().count().next()).result:d}")

Number of vertexes present in graph: 653


In [20]:
print(f"Number of edges present in graph: {gqr(graph_db.g.E().count().next()).result:d}")

Number of edges present in graph: 9075
